In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
import torchtext
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import re
from collections import Counter
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 能用gpu则用gpu

# 1.准备数据

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, stopwords, debug=True):
        df = pd.read_csv(file_path)
        df = df.dropna().reset_index(drop=True)
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        counter = Counter()
        sentences = []
        for title in tqdm(df['title']):
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        self.vocab.set_default_index(self.vocab['<unk>'])

        self.inputs = [self.vocab.lookup_indices(tokens) for tokens in sentences]
        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [3]:
file_path = '../data/THUCNews/train.csv'
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='zh_core_web_sm')
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
dataset = MyDataset(file_path, tokenizer, stopwords, debug=True)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [4]:
def collate_fn(batch_data):
    return pad_sequence([x for x, y in batch_data], padding_value=1), torch.tensor([y for x, y in batch_data]).unsqueeze(1)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# 2.构建模型

In [5]:
class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class, n_hidden):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # rnn层，深度为1
        self.rnn = nn.RNN(embed_size, n_hidden, 1, batch_first=True)
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # 输出头
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = x.permute(1, 0)
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        out, _ = self.rnn(x)  # [batch_size * 句子长度 * n_hidden]
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc(out[:,-1,:])  # 全连接输出头，[batch_size * n_class]
        return logits

In [6]:
model = TextRNN(vocab_size=len(dataset.vocab), embed_size=256,
                n_class=dataset.n_class, n_hidden=256)
print(model)

TextRNN(
  (embedding): Embedding(8296, 256)
  (rnn): RNN(256, 256, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=14, bias=True)
)


# 3.训练模型

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model.train()
for epoch in range(50):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

epoch: 1 , loss: 2.174243688583374
epoch: 2 , loss: 2.2110953330993652
epoch: 3 , loss: 2.2244646549224854
epoch: 4 , loss: 1.8100379705429077
epoch: 5 , loss: 2.5221920013427734
epoch: 6 , loss: 0.8147572875022888
epoch: 7 , loss: 1.1269183158874512
epoch: 8 , loss: 1.3657796382904053
epoch: 9 , loss: 0.7826325297355652
epoch: 10 , loss: 0.9504817724227905
epoch: 11 , loss: 1.0735821723937988
epoch: 12 , loss: 1.4629770517349243
epoch: 13 , loss: 0.9995056390762329
epoch: 14 , loss: 0.08523799479007721
epoch: 15 , loss: 0.23698575794696808
epoch: 16 , loss: 1.2046316862106323
epoch: 17 , loss: 0.17610621452331543
epoch: 18 , loss: 0.31034454703330994
epoch: 19 , loss: 0.03145840764045715
epoch: 20 , loss: 0.1438242644071579
epoch: 21 , loss: 0.1259716898202896
epoch: 22 , loss: 0.09876754879951477
epoch: 23 , loss: 0.026621876284480095
epoch: 24 , loss: 0.05967681109905243
epoch: 25 , loss: 0.05308494344353676
epoch: 26 , loss: 0.09321297705173492
epoch: 27 , loss: 0.00439871707931160

# 4.预测

In [8]:
model.eval()
df_train = pd.read_csv('../data/THUCNews/train.csv')
df_test = pd.read_csv('../data/THUCNews/test.csv')

In [10]:
for i, row in df_test.sample(10).iterrows():
    title = row['title']
    actual = row['class']
    title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
    tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
    inputs = dataset.vocab.lookup_indices(tokens)
    inputs = torch.LongTensor(inputs).unsqueeze(1).to(device)
    predict = model(inputs)
    predict_class = dict(zip(df_train['label'], df_train['class']))[predict.max(1)[1].item()]    
    print(' '.join(tokens), '||| actual:', actual, ', predict:', predict_class)

女 司机 穿 睡衣 载客 跑 生意 遭 抢劫 ||| actual: 社会 , predict: 体育
美国 遣返 名 间谍案 俄罗斯人 ||| actual: 时政 , predict: 股票
港股 认股 证成 交额 亿 元 占 大市 ||| actual: 股票 , predict: 股票
沪 铜 连续 跌 停多 头 两 日 浮亏 超亿 ||| actual: 财经 , predict: 社会
明星 惨遭 金融 摧残 李嘉欣 巨亏 亿 图 ||| actual: 股票 , predict: 体育
全筑 集团 万科 全 装修 合作 驶入 快车道 ||| actual: 家居 , predict: 财经
下午 茶 喝 出 优美 瘦曲线 组图 ||| actual: 时尚 , predict: 社会
波 震荡 上涨 牛市 正在 延续 ||| actual: 股票 , predict: 科技
李嘉欣 婚讯 故弄玄虚亿 元 身家 沉没 股 海图 ||| actual: 娱乐 , predict: 股票
勒布朗 炫丽 三 双 获 赛季 高 评价 昔日 废柴 竟 并列 皇帝 ||| actual: 体育 , predict: 体育
